In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score,accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
production_df=pd.read_csv('crop_production.csv',index_col=False)
rainfal_df=pd.read_csv('New_Rainfall.csv',index_col=False)
#Clean Production file.Drop missing values
production_df=production_df.dropna()

# Group the data by "State"
grouped = rainfal_df.groupby("State_Name")

# Fill missing values within each group
rainfal_df["ANNUAL"] = grouped["ANNUAL"].transform(lambda x: x.fillna(x.mean()))

rainfal_df['Jan-Feb '] = grouped['Jan-Feb '].transform(lambda x: x.fillna(x.mean()))
rainfal_df['Jun-Sep '] = grouped['Jun-Sep '].transform(lambda x: x.fillna(x.mean()))
rainfal_df['Oct-Dec   '] = grouped['Oct-Dec   '].transform(lambda x: x.fillna(x.mean()))
merged_df=pd.merge(rainfal_df,production_df,on=['Crop_Year','State_Name'],how='inner')
merged_df=merged_df.drop(['District_Name'],axis=1)
#we are droping districts to avoid unnessesary extra columns after getdummies function

In [41]:
merged_df['Production/Area']=merged_df['Production']/merged_df['Area']
merged_df

,State_Name,Crop_Year,ANNUAL,Jan-Feb,Mar-May,Jun-Sep,Oct-Dec,Season,Crop,Area,Production,Production/Area
0,Andaman and Nicobar Islands,2000,2763.2,112.0,812.2,1244.2,594.7,Kharif,Arecanut,1254.0,2000.0,1.594896
1,Andaman and Nicobar Islands,2000,2763.2,112.0,812.2,1244.2,594.7,Kharif,Other Kharif pulses,2.0,1.0,0.500000
2,Andaman and Nicobar Islands,2000,2763.2,112.0,812.2,1244.2,594.7,Kharif,Rice,102.0,321.0,3.147059
3,Andaman and Nicobar Islands,2000,2763.2,112.0,812.2,1244.2,594.7,Whole Year,Banana,176.0,641.0,3.642045
4,Andaman and Nicobar Islands,2000,2763.2,112.0,812.2,1244.2,594.7,Whole Year,Cashewnut,720.0,165.0,0.229167
...,...,...,...,...,...,...,...,...,...,...,...,...
209193,West Bengal,2014,2322.6,26.9,394.2,1862.6,38.9,Summer,Rice,306.0,801.0,2.617647
209194,West Bengal,2014,2322.6,26.9,394.2,1862.6,38.9,Summer,Sesamum,627.0,463.0,0.738437
209195,West Bengal,2014,2322.6,26.9,394.2,1862.6,38.9,Whole Year,Sugarcane,324.0,16250.0,50.154321
209196,West Bengal,2014,2322.6,26.9,394.2,1862.6,38.9,Winter,Rice,279151.0,597899.0,2.141848


In [52]:
Rice=merged_df[merged_df['Crop']=='Rice']
Rice

,State_Name,Crop_Year,ANNUAL,Jan-Feb,Mar-May,Jun-Sep,Oct-Dec,Season,Crop,Area,Production,Production/Area
2,Andaman and Nicobar Islands,2000,2763.2,112.0,812.2,1244.2,594.7,Kharif,Rice,102.0,321.00,3.147059
12,Andaman and Nicobar Islands,2000,2763.2,112.0,812.2,1244.2,594.7,Kharif,Rice,10779.0,31863.00,2.956026
23,Andaman and Nicobar Islands,2001,3080.9,104.7,878.7,1515.4,582.0,Kharif,Rice,83.0,300.00,3.614458
31,Andaman and Nicobar Islands,2001,3080.9,104.7,878.7,1515.4,582.0,Kharif,Rice,9718.0,27033.00,2.781745
37,Andaman and Nicobar Islands,2002,2620.2,10.7,478.4,1525.6,605.5,Kharif,Rice,189.2,510.84,2.700000
...,...,...,...,...,...,...,...,...,...,...,...,...
209156,West Bengal,2014,2322.6,26.9,394.2,1862.6,38.9,Summer,Rice,98286.0,374824.00,3.813605
209160,West Bengal,2014,2322.6,26.9,394.2,1862.6,38.9,Winter,Rice,94963.0,273137.00,2.876247
209165,West Bengal,2014,2322.6,26.9,394.2,1862.6,38.9,Autumn,Rice,264.0,721.00,2.731061
209193,West Bengal,2014,2322.6,26.9,394.2,1862.6,38.9,Summer,Rice,306.0,801.00,2.617647


In [79]:
Rice['Season'].unique()

array(['Kharif     ', 'Autumn     ', 'Rabi       ', 'Whole Year ',
       'Summer     ', 'Winter     '], dtype=object)

In [44]:
#Rice production per season
Rice_perseason=Rice.groupby('Season')
Rice_perseason['Production'].sum()
#kharip season produces large amount of rice so its ggod to produce it in kharip season

Season
Autumn         4.502062e+07
Kharif         8.145959e+08
Rabi           7.197221e+07
Summer         1.073641e+08
Whole Year     1.051984e+07
Winter         3.275772e+08
Name: Production, dtype: float64

In [80]:
#Rice production per State
Rice_State=Rice.groupby('State_Name')
Rice_produce=dict(Rice_State['Production'].sum())
Rice_Area=dict(Rice_State['Area'].sum())
sorted_produce=sorted(Rice_produce.items(), key=lambda item: item[1])
sorted_produce
Rice_Area
#overall rice produce is maximum in west bengol and minimum in chandigarh

{'Andaman and Nicobar Islands': 77714.93,
 'Andhra Pradesh': 38568565.0,
 'Arunachal Pradesh': 1851432.0,
 'Assam': 37125826.0,
 'Bihar': 50335716.0,
 'Chandigarh': 650.0,
 'Chhattisgarh': 58259645.0,
 'Dadra and Nagar Haveli': 202071.0,
 'Goa': 530694.0,
 'Gujarat': 8983000.0,
 'Haryana': 14289149.0,
 'Himachal Pradesh': 798141.0,
 'Jammu and Kashmir ': 2038060.0,
 'Jharkhand': 7726361.13,
 'Karnataka': 20636355.0,
 'Kerala': 3810518.24,
 'Madhya Pradesh': 23767165.0,
 'Maharashtra': 21351136.0,
 'Manipur': 1304490.0,
 'Meghalaya': 1568924.0,
 'Mizoram': 544113.0,
 'Nagaland': 2511840.0,
 'Odisha': 72687520.0,
 'Puducherry': 335131.0,
 'Punjab': 40695000.0,
 'Rajasthan': 1353673.0,
 'Sikkim': 183166.0,
 'Tamil Nadu': 25500280.0,
 'Telangana ': 19991009.0,
 'Tripura': 3800840.0,
 'Uttar Pradesh': 86580851.0,
 'Uttarakhand': 4437208.0,
 'West Bengal': 84038642.0}

In [81]:
#list(np.mean(Rice_State['Production/Area']))

In [51]:
Rice_State=Rice.groupby('State_Name')
Rice_produce_perArea=dict(Rice_State['Production/Area'].sum())
sorted_produce=sorted(Rice_produce_perArea.items(), key=lambda item: item[1])
sorted_produce

[('Andaman and Nicobar Islands', 49.38135862078335),
 ('Dadra and Nagar Haveli', 53.70656836313445),
 ('Chandigarh', 55.15),
 ('Goa', 94.45440422953442),
 ('Mizoram', 110.53134149315567),
 ('Sikkim', 135.56056132884223),
 ('Manipur', 162.4757819330867),
 ('Himachal Pradesh', 170.30895720280782),
 ('Tripura', 182.2617684505702),
 ('Jammu and Kashmir ', 188.65870164333091),
 ('Jharkhand', 221.52751481053556),
 ('Arunachal Pradesh', 281.84938487523846),
 ('Puducherry', 284.4895315810863),
 ('Nagaland', 314.37107162479657),
 ('Rajasthan', 360.05206360445345),
 ('Chhattisgarh', 417.9320433181832),
 ('Uttarakhand', 461.23432666690286),
 ('Meghalaya', 496.2352320085237),
 ('Gujarat', 645.2223665447805),
 ('Madhya Pradesh', 665.8876043200681),
 ('Haryana', 690.1352074449007),
 ('Telangana ', 721.7134074747955),
 ('Maharashtra', 760.8685680854862),
 ('Punjab', 1083.9607480055802),
 ('Andhra Pradesh', 1194.9817547833413),
 ('Kerala', 1400.4104333111886),
 ('Tamil Nadu', 1429.7669875944719),
 ('O

In [54]:
Rice[Rice['State_Name']=='Uttar Pradesh']

,State_Name,Crop_Year,ANNUAL,Jan-Feb,Mar-May,Jun-Sep,Oct-Dec,Season,Crop,Area,Production,Production/Area
168405,Uttar Pradesh,2000,771.4,31.7,27.4,712.2,0.1,Kharif,Rice,1322.0,2502.0,1.892587
168429,Uttar Pradesh,2000,771.4,31.7,27.4,712.2,0.1,Kharif,Rice,34648.0,68335.0,1.972264
168449,Uttar Pradesh,2000,771.4,31.7,27.4,712.2,0.1,Kharif,Rice,210540.0,385279.0,1.829956
168473,Uttar Pradesh,2000,771.4,31.7,27.4,712.2,0.1,Kharif,Rice,116502.0,288186.0,2.473657
168496,Uttar Pradesh,2000,771.4,31.7,27.4,712.2,0.1,Kharif,Rice,28486.0,66368.0,2.329846
...,...,...,...,...,...,...,...,...,...,...,...,...
196281,Uttar Pradesh,2014,486.9,77.7,39.0,339.4,30.8,Kharif,Rice,29426.0,52290.0,1.777000
196309,Uttar Pradesh,2014,486.9,77.7,39.0,339.4,30.8,Kharif,Rice,85103.0,151568.0,1.780995
196342,Uttar Pradesh,2014,486.9,77.7,39.0,339.4,30.8,Kharif,Rice,96435.0,161818.0,1.678001
196364,Uttar Pradesh,2014,486.9,77.7,39.0,339.4,30.8,Summer,Rice,1.0,2.0,2.000000


In [58]:
up=production_df[production_df['State_Name']=='Uttar Pradesh']
up[up['Crop']=='Rice']
up['Production/Area']=up['Production']/up['Area']
np.mean(up['Production/Area'])

5.284643939921226

In [14]:
def factorial(n):
    if(n>0):
        fact=1
        for i in range(1,n+1):
            fact=fact*i
        print(fact)
        return fact
    else:
        return 'enter proper no'

In [15]:
factorial(5)

120


120

In [17]:
#Factorial till nth no
n=5
for j in range(n+1):
    factorial(j)

1
2
6
24
120


In [26]:
def getFactorial(num):
    
    if num == 0:
             return 1
    return num * getFactorial(num - 1)
getFactorial(6)

720

In [33]:
#reverse no
n=12546
while(n!=0):
    
    reminder=int(n%10)
    n=int(n/10)
    print(reminder,end='')

64521

In [36]:
#reverse a no
num=12548
x=list(str(num))[::-1]
int(''.join(x))

84521

In [47]:
#leap year 
def leap_year(n):
    if(n%4==0):
        if(n%100==0):
            
            if(n%400==0):
                print('Its leap year with century')
        else:
            print('its leap year')
    else:
        print('its not a leap year')
        

In [49]:
leap_year(2007)

its not a leap year
